# Métodos Basados en Densidad - Ejercicios

---
Este cuaderno contiene ejercicios prácticos para consolidar los conceptos del Módulo 4.


## Configuración Inicial

Ejecuta esta celda antes de comenzar los ejercicios.

In [ ]:
# Instalación de HDBSCAN
!pip install hdbscan -q

In [ ]:
# Bibliotecas necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.metrics import confusion_matrix

# HDBSCAN
import hdbscan

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Configuración completada.")

---

## Ejercicio 1: Comprensión de los Conceptos de DBSCAN

### Contexto

Para comprender DBSCAN es fundamental entender los conceptos de epsilon-vecindad y la clasificación de puntos en núcleo, borde y ruido.

### Datos

In [ ]:
# Dataset pequeño para análisis manual
X_ej1 = np.array([
    [1.0, 1.0],   # P0
    [1.3, 1.2],   # P1
    [0.9, 1.4],   # P2
    [1.2, 0.8],   # P3
    [1.5, 1.1],   # P4
    [4.0, 4.0],   # P5
    [4.2, 4.3],   # P6
    [4.1, 3.8],   # P7
    [2.5, 2.5],   # P8 - punto intermedio
    [7.0, 1.0]    # P9 - outlier
])

# Visualización
plt.figure(figsize=(10, 8))
plt.scatter(X_ej1[:, 0], X_ej1[:, 1], c='steelblue', s=150, edgecolors='w', linewidths=2)
for i, (x, y) in enumerate(X_ej1):
    plt.annotate(f'P{i}', (x + 0.15, y + 0.15), fontsize=12, fontweight='bold')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset para Ejercicio 1')
plt.grid(True, alpha=0.3)
plt.show()

### Tarea 1.1: Cálculo de Vecindades

Dado eps=0.6 y min_samples=3, calcule la epsilon-vecindad de cada punto y determine cuántos vecinos tiene cada uno.

In [ ]:
# Parámetros
eps = 0.6
min_samples = 3

# TODO: Calcular las vecindades usando NearestNeighbors
# Use radius_neighbors con el radio eps

# Su código aquí
nn = None  # Reemplazar
vecindades = None  # Reemplazar

# Mostrar el número de vecinos de cada punto
if vecindades is not None:
    print(f"Epsilon-vecindades (eps={eps}):")
    print("-" * 40)
    for i, vecinos in enumerate(vecindades):
        print(f"P{i}: {len(vecinos)} vecinos -> {list(vecinos)}")

### Tarea 1.2: Clasificación de Puntos

Clasifique cada punto como núcleo, borde o ruido según los criterios de DBSCAN.

In [ ]:
# TODO: Clasificar cada punto
# Punto núcleo: |N_eps(p)| >= min_samples
# Punto borde: no es núcleo pero está en la vecindad de un núcleo
# Ruido: no es núcleo ni está en la vecindad de ningún núcleo

tipos = []  # Lista para almacenar el tipo de cada punto

# Su código aquí


# Mostrar clasificación
if len(tipos) > 0:
    print("Clasificación de puntos:")
    print("-" * 40)
    for i, tipo in enumerate(tipos):
        print(f"P{i}: {tipo}")

### Tarea 1.3: Verificación con DBSCAN de Scikit-learn

Aplique DBSCAN y compare los resultados con su clasificación manual.

In [ ]:
# TODO: Aplicar DBSCAN y analizar los resultados
# Compare las etiquetas asignadas con su clasificación manual

# Su código aquí
dbscan = None  # Reemplazar
labels = None  # Reemplazar

if labels is not None:
    print("Resultados de DBSCAN:")
    print("-" * 40)
    for i, label in enumerate(labels):
        estado = "Ruido" if label == -1 else f"Cluster {label}"
        print(f"P{i}: {estado}")

    # Identificar puntos núcleo
    print(f"\nÍndices de puntos núcleo: {dbscan.core_sample_indices_}")

### Pregunta de Reflexión 1

Analice los resultados:

1. ¿Cuántos clusters se formaron? ¿Coincide con lo que observa visualmente?
2. ¿El punto P8 (intermedio) fue clasificado como núcleo, borde o ruido? ¿Por qué?
3. ¿Qué pasaría si aumentamos eps a 1.0? ¿Y si lo reducimos a 0.4?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 2: Selección de Parámetros con k-Distance Graph

### Contexto

El método del k-distance graph es fundamental para seleccionar un valor apropiado de eps. En este ejercicio, aplicará esta técnica a diferentes datasets.

### Datos

In [ ]:
# Generar dataset con estructura clara
X_ej2, y_ej2 = make_blobs(
    n_samples=300,
    centers=4,
    cluster_std=[0.5, 0.8, 0.6, 0.7],
    random_state=RANDOM_STATE
)

# Añadir algunos outliers
outliers = np.random.uniform(low=-12, high=12, size=(15, 2))
X_ej2_noisy = np.vstack([X_ej2, outliers])
y_ej2_noisy = np.hstack([y_ej2, [-1]*15])

plt.figure(figsize=(10, 6))
plt.scatter(X_ej2_noisy[:300, 0], X_ej2_noisy[:300, 1], c=y_ej2, cmap='viridis',
            edgecolors='w', s=50, alpha=0.7)
plt.scatter(X_ej2_noisy[300:, 0], X_ej2_noisy[300:, 1], c='red', marker='x',
            s=100, label='Outliers')
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset para Ejercicio 2 (4 clusters + outliers)')
plt.legend()
plt.show()

### Tarea 2.1: Implementar la Función k-Distance

Implemente una función que calcule y grafique las k-distancias ordenadas.

In [ ]:
def calcular_k_distance(X, k):
    """
    Calcula la distancia al k-ésimo vecino más cercano para cada punto.

    Parámetros:
    -----------
    X : array-like de forma (n_samples, n_features)
        Datos de entrada.
    k : int
        Número de vecinos a considerar.

    Retorna:
    --------
    k_distances : array de forma (n_samples,)
        Distancias al k-ésimo vecino, ordenadas de menor a mayor.
    """
    # TODO: Implementar la función
    # 1. Crear un objeto NearestNeighbors con n_neighbors=k
    # 2. Ajustar a los datos X
    # 3. Obtener las distancias con kneighbors
    # 4. Extraer la distancia al k-ésimo vecino (última columna)
    # 5. Ordenar de menor a mayor

    # Su código aquí
    k_distances = None  # Reemplazar

    return k_distances

# Verificar la función
k = 5
k_dist = calcular_k_distance(X_ej2_noisy, k)
if k_dist is not None:
    print(f"k-distances calculadas: {len(k_dist)} valores")
    print(f"Rango: [{k_dist.min():.3f}, {k_dist.max():.3f}]")

### Tarea 2.2: Graficar k-Distance y Encontrar el Codo

Genere el gráfico de k-distancias e identifique el punto de inflexión (codo).

In [ ]:
# TODO: Graficar k-distance para diferentes valores de k
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

k_values = [3, 5, 10]

for idx, k in enumerate(k_values):
    ax = axes[idx]

    # TODO: Calcular k-distances y graficar
    # Su código aquí

    ax.set_xlabel('Puntos ordenados')
    ax.set_ylabel(f'Distancia al {k}-ésimo vecino')
    ax.set_title(f'k-Distance Graph (k={k})')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Tarea 2.3: Aplicar DBSCAN con el eps Seleccionado

Use el valor de eps identificado en el gráfico anterior.

In [ ]:
# TODO: Aplicar DBSCAN con el eps identificado
# Pruebe con el valor del codo que identificó

eps_optimo = None  # Reemplazar con su valor
min_samples = 5

if eps_optimo is not None:
    # Su código aquí

    # Visualizar resultados
    pass

### Pregunta de Reflexión 2

Analice el proceso de selección de parámetros:

1. ¿El valor de eps identificado permitió detectar correctamente los 4 clusters?
2. ¿Cuántos outliers fueron detectados? ¿Coincide con los 15 que añadimos?
3. ¿Cómo afecta el valor de k a la forma de la curva y al eps sugerido?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 3: Comparación de Algoritmos en Estructuras No Convexas

### Contexto

Una ventaja clave de los métodos basados en densidad es su capacidad para detectar clusters de forma arbitraria. Compare el rendimiento de K-Means, DBSCAN y HDBSCAN.

### Datos

In [ ]:
# Generar datasets con diferentes estructuras
n_samples = 500

# Dataset 1: Lunas
X_lunas, y_lunas = make_moons(n_samples=n_samples, noise=0.05, random_state=RANDOM_STATE)

# Dataset 2: Círculos concéntricos
X_circulos, y_circulos = make_circles(n_samples=n_samples, noise=0.05, factor=0.5,
                                       random_state=RANDOM_STATE)

# Dataset 3: Forma de S con ruido
t = np.linspace(0, 2*np.pi, n_samples//2)
x1 = np.sin(t) + np.random.normal(0, 0.1, len(t))
y1 = t + np.random.normal(0, 0.1, len(t))
x2 = np.sin(t) + 2 + np.random.normal(0, 0.1, len(t))
y2 = t + np.random.normal(0, 0.1, len(t))
X_s = np.vstack([np.column_stack([x1, y1]), np.column_stack([x2, y2])])
y_s = np.array([0]*len(t) + [1]*len(t))

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
datasets = [(X_lunas, y_lunas, 'Lunas'), (X_circulos, y_circulos, 'Círculos'),
            (X_s, y_s, 'Forma de S')]

for ax, (X, y, titulo) in zip(axes, datasets):
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='w', s=30)
    ax.set_title(f'{titulo} (Ground Truth)')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.tight_layout()
plt.show()

### Tarea 3.1: Aplicar K-Means a Cada Dataset

In [ ]:
# TODO: Aplicar K-Means (k=2) a cada dataset y calcular ARI
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

resultados_kmeans = []

for idx, (X, y_true, titulo) in enumerate(datasets):
    ax = axes[idx]

    # TODO: Aplicar K-Means
    # Su código aquí

    ax.set_title(f'{titulo}')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('K-Means (k=2)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Tarea 3.2: Aplicar DBSCAN con Parámetros Apropiados

In [ ]:
# TODO: Aplicar DBSCAN a cada dataset
# Seleccione eps apropiado para cada uno (puede usar k-distance)

# Parámetros sugeridos para explorar:
# Lunas: eps entre 0.1 y 0.3
# Círculos: eps entre 0.1 y 0.3
# Forma S: eps entre 0.2 y 0.5

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# TODO: Definir parámetros para cada dataset
params_dbscan = [
    {'eps': None, 'min_samples': 5},  # Lunas - Reemplazar eps
    {'eps': None, 'min_samples': 5},  # Círculos - Reemplazar eps
    {'eps': None, 'min_samples': 5}   # Forma S - Reemplazar eps
]

resultados_dbscan = []

for idx, (X, y_true, titulo) in enumerate(datasets):
    ax = axes[idx]

    # TODO: Aplicar DBSCAN con los parámetros definidos
    # Su código aquí

    ax.set_title(f'{titulo}')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('DBSCAN', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Tarea 3.3: Aplicar HDBSCAN

In [ ]:
# TODO: Aplicar HDBSCAN a cada dataset
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

resultados_hdbscan = []

for idx, (X, y_true, titulo) in enumerate(datasets):
    ax = axes[idx]

    # TODO: Aplicar HDBSCAN con min_cluster_size=15
    # Su código aquí

    ax.set_title(f'{titulo}')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('HDBSCAN', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Tarea 3.4: Tabla Comparativa de Resultados

In [ ]:
# TODO: Crear una tabla comparativa con los ARI de cada algoritmo

# Su código aquí para generar la tabla
# Formato sugerido:
# Dataset     | K-Means | DBSCAN | HDBSCAN
# Lunas       |   xxx   |  xxx   |   xxx
# Círculos    |   xxx   |  xxx   |   xxx
# Forma S     |   xxx   |  xxx   |   xxx

### Pregunta de Reflexión 3

Basándose en los resultados:

1. ¿Qué algoritmo tuvo mejor rendimiento general? ¿Por qué?
2. ¿En qué dataset K-Means tuvo peor rendimiento? Explique la razón.
3. ¿HDBSCAN requirió ajuste de parámetros? ¿Es esto una ventaja o desventaja?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 4: HDBSCAN - Características Avanzadas

### Contexto

HDBSCAN proporciona información adicional como probabilidades de pertenencia y outlier scores. Explore estas características.

### Datos

In [ ]:
# Dataset con clusters de diferente densidad y outliers
np.random.seed(RANDOM_STATE)

# Cluster denso
cluster_denso = np.random.randn(150, 2) * 0.3 + np.array([0, 0])

# Cluster medio
cluster_medio = np.random.randn(100, 2) * 0.6 + np.array([4, 4])

# Cluster disperso
cluster_disperso = np.random.randn(80, 2) * 1.2 + np.array([-4, 4])

# Outliers
outliers = np.array([
    [6, -2], [-6, -2], [0, 7], [7, 7], [-7, 0],
    [3, -3], [-3, -3], [8, 2], [-8, 2], [0, -5]
])

X_ej4 = np.vstack([cluster_denso, cluster_medio, cluster_disperso, outliers])
y_ej4 = np.array([0]*150 + [1]*100 + [2]*80 + [-1]*10)

plt.figure(figsize=(10, 8))
for label in [0, 1, 2]:
    mask = y_ej4 == label
    plt.scatter(X_ej4[mask, 0], X_ej4[mask, 1], label=f'Cluster {label}',
                edgecolors='w', s=50, alpha=0.7)
plt.scatter(outliers[:, 0], outliers[:, 1], c='red', marker='x', s=100,
            label='Outliers', linewidths=2)
plt.xlabel('Característica 1')
plt.ylabel('Característica 2')
plt.title('Dataset con densidad variable y outliers')
plt.legend()
plt.show()

### Tarea 4.1: Aplicar HDBSCAN y Extraer Probabilidades

In [ ]:
# TODO: Aplicar HDBSCAN y extraer probabilidades y outlier scores

# Su código aquí
clusterer = None  # Reemplazar
labels_hdb = None  # Reemplazar
probabilidades = None  # Reemplazar (clusterer.probabilities_)
outlier_scores = None  # Reemplazar (clusterer.outlier_scores_)

if labels_hdb is not None:
    n_clusters = len(set(labels_hdb)) - (1 if -1 in labels_hdb else 0)
    n_ruido = (labels_hdb == -1).sum()
    print(f"Clusters encontrados: {n_clusters}")
    print(f"Puntos de ruido: {n_ruido}")

### Tarea 4.2: Visualizar Probabilidades de Pertenencia

In [ ]:
# TODO: Crear visualización de probabilidades
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Subplot 1: Clusters coloreados
ax = axes[0]
# Su código aquí

# Subplot 2: Probabilidades de pertenencia
ax = axes[1]
# Su código aquí (usar colormap para mostrar probabilidades)

plt.tight_layout()
plt.show()

### Tarea 4.3: Análisis de Outlier Scores

In [ ]:
# TODO: Identificar los puntos con mayor outlier score
# Compare con los outliers reales

if outlier_scores is not None:
    # TODO: Encontrar los 10 puntos con mayor outlier score
    # Su código aquí

    # Visualizar
    fig, ax = plt.subplots(figsize=(10, 8))
    # Su código aquí

    plt.show()

### Pregunta de Reflexión 4

Analice las características de HDBSCAN:

1. ¿Las probabilidades de pertenencia son más bajas en los bordes de los clusters? ¿Por qué?
2. ¿HDBSCAN detectó correctamente los 10 outliers? ¿Hubo falsos positivos o negativos?
3. ¿Cómo podría usar los outlier scores en una aplicación de detección de fraude?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 5: Caso Práctico - Segmentación de Clientes

### Contexto

Aplique métodos basados en densidad a un problema de segmentación de clientes usando datos de comportamiento de compra.

### Datos

In [ ]:
# Simular datos de clientes
np.random.seed(RANDOM_STATE)

n_clientes = 400

# Segmento 1: Clientes frecuentes de bajo gasto
seg1_frecuencia = np.random.normal(25, 3, 100)  # Visitas/mes
seg1_gasto = np.random.normal(50, 10, 100)       # Gasto promedio

# Segmento 2: Clientes ocasionales de alto gasto
seg2_frecuencia = np.random.normal(5, 2, 120)
seg2_gasto = np.random.normal(200, 30, 120)

# Segmento 3: Clientes premium (alta frecuencia, alto gasto)
seg3_frecuencia = np.random.normal(20, 4, 80)
seg3_gasto = np.random.normal(300, 40, 80)

# Segmento 4: Clientes inactivos (cluster disperso)
seg4_frecuencia = np.random.normal(2, 1.5, 80)
seg4_gasto = np.random.normal(30, 20, 80)

# Clientes anómalos
anomalos_frecuencia = np.array([40, 1, 35, 50, 0.5])
anomalos_gasto = np.array([500, 600, 20, 400, 450])

# Combinar datos
frecuencia = np.concatenate([seg1_frecuencia, seg2_frecuencia, seg3_frecuencia,
                             seg4_frecuencia, anomalos_frecuencia])
gasto = np.concatenate([seg1_gasto, seg2_gasto, seg3_gasto, seg4_gasto, anomalos_gasto])

# Asegurar valores positivos
frecuencia = np.clip(frecuencia, 0.1, None)
gasto = np.clip(gasto, 10, None)

# Crear DataFrame
df_clientes = pd.DataFrame({
    'frecuencia_mensual': frecuencia,
    'gasto_promedio': gasto,
    'segmento_real': [0]*100 + [1]*120 + [2]*80 + [3]*80 + [-1]*5
})

print(f"Dataset: {len(df_clientes)} clientes")
print(f"\nEstadísticas descriptivas:")
print(df_clientes[['frecuencia_mensual', 'gasto_promedio']].describe())

### Tarea 5.1: Preprocesamiento y Exploración

In [ ]:
# TODO: Estandarizar los datos y visualizar

# Su código aquí
scaler = StandardScaler()
X_clientes = None  # Reemplazar

# Visualización
if X_clientes is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Datos originales
    ax = axes[0]
    # Su código aquí

    # Datos estandarizados
    ax = axes[1]
    # Su código aquí

    plt.tight_layout()
    plt.show()

### Tarea 5.2: Selección de Parámetros para DBSCAN

In [ ]:
# TODO: Usar k-distance para determinar eps

# Su código aquí

### Tarea 5.3: Aplicar y Comparar Algoritmos

In [ ]:
# TODO: Aplicar K-Means, DBSCAN y HDBSCAN
# Comparar resultados

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Ground Truth
ax = axes[0, 0]
# Su código aquí

# K-Means
ax = axes[0, 1]
# Su código aquí

# DBSCAN
ax = axes[1, 0]
# Su código aquí

# HDBSCAN
ax = axes[1, 1]
# Su código aquí

plt.tight_layout()
plt.show()

### Tarea 5.4: Perfilado de Segmentos

In [ ]:
# TODO: Crear perfiles de los segmentos encontrados por HDBSCAN
# Calcular estadísticas por segmento

# Su código aquí

### Pregunta de Reflexión 5

Analice los resultados de la segmentación:

1. ¿Qué algoritmo identificó mejor los segmentos de clientes? ¿Por qué?
2. ¿Se detectaron correctamente los clientes anómalos? ¿Qué caracteriza a estos clientes?
3. ¿Cómo describiría cada segmento en términos de negocio? ¿Qué estrategias de marketing sugeriría?

*Escriba su respuesta aquí:*

...

---

## Ejercicio 6: Sensibilidad a la Dimensionalidad

### Contexto

Los métodos basados en densidad pueden verse afectados por la alta dimensionalidad. Explore este fenómeno.

### Datos

In [ ]:
# Generar datos en diferentes dimensionalidades
def generar_datos_nd(n_samples, n_features, n_clusters, random_state=42):
    """Genera datos con clusters en n dimensiones."""
    X, y = make_blobs(n_samples=n_samples, n_features=n_features,
                      centers=n_clusters, random_state=random_state)
    return X, y

# Crear datasets de diferentes dimensionalidades
dimensiones = [2, 5, 10, 20, 50]
datasets_dim = {}

for d in dimensiones:
    X, y = generar_datos_nd(n_samples=300, n_features=d, n_clusters=3)
    datasets_dim[d] = (X, y)
    print(f"Dimensión {d}: {X.shape}")

### Tarea 6.1: Analizar Distribución de Distancias

In [ ]:
# TODO: Calcular y visualizar la distribución de distancias en cada dimensión
from scipy.spatial.distance import pdist

fig, axes = plt.subplots(1, len(dimensiones), figsize=(18, 4))

for idx, d in enumerate(dimensiones):
    ax = axes[idx]
    X, _ = datasets_dim[d]

    # TODO: Calcular distancias y graficar histograma
    # Su código aquí

    ax.set_title(f'd = {d}')
    ax.set_xlabel('Distancia')

axes[0].set_ylabel('Frecuencia')
plt.suptitle('Distribución de distancias por dimensionalidad', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Tarea 6.2: Rendimiento de DBSCAN vs Dimensionalidad

In [ ]:
# TODO: Aplicar DBSCAN a cada dataset y medir rendimiento
# Use un eps fijo y observe cómo cambia el resultado

resultados_dim = []

for d in dimensiones:
    X, y_true = datasets_dim[d]

    # TODO: Aplicar DBSCAN
    # Calcular ARI y número de clusters/ruido

    # Su código aquí
    pass

# Mostrar resultados en tabla

### Pregunta de Reflexión 6

Analice el efecto de la dimensionalidad:

1. ¿Cómo cambia la distribución de distancias al aumentar la dimensionalidad?
2. ¿Por qué DBSCAN tiene problemas en alta dimensionalidad?
3. ¿Qué estrategias podría usar para aplicar clustering basado en densidad a datos de alta dimensionalidad?

*Escriba su respuesta aquí:*

...

---

## Referencias

- Ester, M., Kriegel, H. P., Sander, J., & Xu, X. (1996). A density-based algorithm for discovering clusters in large spatial databases with noise. *KDD*, 96(34), 226-231.
- Campello, R. J., Moulavi, D., & Sander, J. (2013). Density-based clustering based on hierarchical density estimates. *PAKDD*, 160-172.
- Scikit-learn documentation: https://scikit-learn.org/stable/modules/clustering.html
- HDBSCAN documentation: https://hdbscan.readthedocs.io/

---



# EOF (End Of File)